In [20]:
input_pdf = "/Users/d062334/Downloads/Vodafone-India-Idea-Presentation.pdf"
input_pdf = "/Users/d062334/Downloads/20050421-smoothing-tutorial.pdf"

In [21]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer
import math


def createPDFDoc(fpath):
    fp = open(fpath, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser, password='')
    if not document.is_extractable:
        raise "Not extractable"
    else:
        return document


def createDeviceInterpreter():
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    return device, interpreter


def parse_obj(objs):
    string_fontsize = []
    for obj in objs:
        if isinstance(obj, pdfminer.layout.LTTextBox):
            for o in obj._objs:
                if isinstance(o,pdfminer.layout.LTTextLine):
                    text=o.get_text()
                    if text.strip():
                        old_size = 0
                        string = ""
                        for c in  o._objs:
                            try:
                                string += c._text
                                old_size = c.size
                            except:
                                pass
                            if isinstance(c, pdfminer.layout.LTChar):
                                pass
                        string_fontsize.append({"string":string, "size":old_size})
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs)
        else:
            pass
    i = 0
    while i < len(string_fontsize):
        if i != 0 and math.floor(string_fontsize[i-1]["size"]) == math.floor(string_fontsize[i]["size"]):
            string_fontsize[i-1]["string"] += string_fontsize[i]["string"]
            del string_fontsize[i]
            i -= 1
        i+=1
        
    return string_fontsize

document=createPDFDoc(input_pdf)
device,interpreter=createDeviceInterpreter()
pages=PDFPage.create_pages(document)
interpreter.process_page(next(pages))
layout = device.get_result()
string_fontsize = parse_obj(layout._objs)
title_index = max(range(len(string_fontsize)), key=lambda index: string_fontsize[index]['size'])
title = string_fontsize[title_index]["string"].replace("\n", " ")
print(title)

NLP Lunch Tutorial: Smoothing 


In [26]:
from pdfrw import PdfReader
title = PdfReader(input_pdf).Info.Title
if title != None: title = title.strip("()").strip()
if title == "": title = None
print(title)

None
